In [ ]:
!pip install PyPortfolioOpt
!pip install backtrader
!pip install quantstats

In [117]:
# Importing the pandas library for data manipulation and analysis
import pandas as pd

# Importing the matplotlib library for creating visualizations
import matplotlib.pyplot as plt

# Importing the backtrader library for backtesting trading strategies
import backtrader as bt

# Importing the numpy library for numerical computing
import numpy as np

# Importing matrix, solvers, and blas from cvxopt library for convex optimization
from cvxopt import matrix, solvers, blas

# Importing the tabulate library for creating tables
from tabulate import tabulate

# Importing the datetime module to work with dates and times
import datetime

# Importing the matplotlib library again (this line is redundant)
import matplotlib.pyplot as plt

# Importing the cvxopt library for convex optimization
import cvxopt as opt

# Importing quantStats for quantitative financial analysis
import quantstats as qs

import math

In [118]:
# Setting the maximum number of columns to display in pandas to None, which means all columns will be displayed
pd.set_option('display.max_columns', None)

# Setting the maximum number of rows to display in pandas to None, which means all rows will be displayed
pd.set_option('display.max_rows', None)

# Using the magic command %matplotlib inline to display matplotlib plots inline in a Jupyter notebook
%matplotlib inline

# Setting the random seed for numpy to 42 for reproducibility
np.random.seed(42)

# Setting the 'show_progress' option for solvers in cvxopt library to False, which means no progress will be shown while solving
solvers.options['show_progress'] = False

In [119]:
df = pd.read_csv('data.csv')
df.head()


,Date,AXISBANK,BAJAJFINSV,BAJAJHLDNG,BAJFINANCE,BANKBARODA,HDFC,BANDHANBNK,GICRE,HDFCAMC,HDFCLIFE,IBULHSGFIN,ICICIBANK,ICICIGI,ICICIPRULI,INDUSINDBK,KOTAKBANK,L&TFH,NIACL,PEL,PFC,PNB,SBILIFE,SBIIN,SRTRANSFIN
0,2010-01-04,140.712021,347.934143,494.222076,16.031584,51.206989,429.764343,NaN,NaN,NaN,NaN,NaN,85.319977,NaN,NaN,133.753540,202.426041,NaN,NaN,252.234650,61.325417,84.943443,NaN,95.066895,421.881561
1,2010-01-05,143.647934,348.708527,498.012665,15.924150,52.686508,431.226868,NaN,NaN,NaN,NaN,NaN,86.130264,NaN,NaN,133.613098,204.430389,NaN,NaN,253.638733,62.669540,86.255913,NaN,95.102180,424.757904
2,2010-01-06,141.236801,340.480804,491.044190,16.153118,53.009216,433.299774,NaN,NaN,NaN,NaN,NaN,86.789574,NaN,NaN,136.796616,206.915863,NaN,NaN,265.181671,62.333515,85.847481,NaN,95.672699,418.562897
3,2010-01-07,142.236725,337.819672,481.012695,16.054960,53.326965,425.232696,NaN,NaN,NaN,NaN,NaN,85.970223,NaN,NaN,133.706726,209.240005,NaN,NaN,265.935547,62.381512,84.668106,NaN,95.137444,431.439545
4,2010-01-08,144.094727,338.787170,478.753693,16.347025,53.540455,417.824463,NaN,NaN,NaN,NaN,NaN,84.762535,NaN,NaN,133.238586,207.948059,NaN,NaN,265.181671,62.033478,84.792007,NaN,94.853218,440.510895


In [120]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

In [121]:
tickersList = df.columns.tolist()
tickersList = tickersList[1:]
tickersList

['AXISBANK',
 'BAJAJFINSV',
 'BAJAJHLDNG',
 'BAJFINANCE',
 'BANKBARODA',
 'HDFC',
 'BANDHANBNK',
 'GICRE',
 'HDFCAMC',
 'HDFCLIFE',
 'IBULHSGFIN',
 'ICICIBANK',
 'ICICIGI',
 'ICICIPRULI',
 'INDUSINDBK',
 'KOTAKBANK',
 'L&TFH',
 'NIACL',
 'PEL',
 'PFC',
 'PNB',
 'SBILIFE',
 'SBIIN',
 'SRTRANSFIN']

In [122]:
df2 = pd.read_csv('prices.csv')

df2['date'] = pd.to_datetime(df2['date'], format='%Y-%m-%d')

# Getting the minimum and maximum dates in the 'date' column
start_date = df2['date'].min()
end_date = df2['date'].max()

# store starting dates for each ticker
start_dates = df2.groupby('ticker')['date'].min()

In [123]:
for s in tickersList:
  df[s] = df[s].fillna(method='bfill')

df.head()

,Date,AXISBANK,BAJAJFINSV,BAJAJHLDNG,BAJFINANCE,BANKBARODA,HDFC,BANDHANBNK,GICRE,HDFCAMC,HDFCLIFE,IBULHSGFIN,ICICIBANK,ICICIGI,ICICIPRULI,INDUSINDBK,KOTAKBANK,L&TFH,NIACL,PEL,PFC,PNB,SBILIFE,SBIIN,SRTRANSFIN
0,2010-01-04,140.712021,347.934143,494.222076,16.031584,51.206989,429.764343,473.382355,409.079987,1770.86731,341.851837,182.552383,85.319977,672.160461,289.724579,133.753540,202.426041,45.2677,353.983917,252.234650,61.325417,84.943443,703.133728,95.066895,421.881561
1,2010-01-05,143.647934,348.708527,498.012665,15.924150,52.686508,431.226868,473.382355,409.079987,1770.86731,341.851837,182.552383,86.130264,672.160461,289.724579,133.613098,204.430389,45.2677,353.983917,253.638733,62.669540,86.255913,703.133728,95.102180,424.757904
2,2010-01-06,141.236801,340.480804,491.044190,16.153118,53.009216,433.299774,473.382355,409.079987,1770.86731,341.851837,182.552383,86.789574,672.160461,289.724579,136.796616,206.915863,45.2677,353.983917,265.181671,62.333515,85.847481,703.133728,95.672699,418.562897
3,2010-01-07,142.236725,337.819672,481.012695,16.054960,53.326965,425.232696,473.382355,409.079987,1770.86731,341.851837,182.552383,85.970223,672.160461,289.724579,133.706726,209.240005,45.2677,353.983917,265.935547,62.381512,84.668106,703.133728,95.137444,431.439545
4,2010-01-08,144.094727,338.787170,478.753693,16.347025,53.540455,417.824463,473.382355,409.079987,1770.86731,341.851837,182.552383,84.762535,672.160461,289.724579,133.238586,207.948059,45.2677,353.983917,265.181671,62.033478,84.792007,703.133728,94.853218,440.510895


In [124]:
df.columns = df.columns.str.strip()

In [125]:
df.head()

,Date,AXISBANK,BAJAJFINSV,BAJAJHLDNG,BAJFINANCE,BANKBARODA,HDFC,BANDHANBNK,GICRE,HDFCAMC,HDFCLIFE,IBULHSGFIN,ICICIBANK,ICICIGI,ICICIPRULI,INDUSINDBK,KOTAKBANK,L&TFH,NIACL,PEL,PFC,PNB,SBILIFE,SBIIN,SRTRANSFIN
0,2010-01-04,140.712021,347.934143,494.222076,16.031584,51.206989,429.764343,473.382355,409.079987,1770.86731,341.851837,182.552383,85.319977,672.160461,289.724579,133.753540,202.426041,45.2677,353.983917,252.234650,61.325417,84.943443,703.133728,95.066895,421.881561
1,2010-01-05,143.647934,348.708527,498.012665,15.924150,52.686508,431.226868,473.382355,409.079987,1770.86731,341.851837,182.552383,86.130264,672.160461,289.724579,133.613098,204.430389,45.2677,353.983917,253.638733,62.669540,86.255913,703.133728,95.102180,424.757904
2,2010-01-06,141.236801,340.480804,491.044190,16.153118,53.009216,433.299774,473.382355,409.079987,1770.86731,341.851837,182.552383,86.789574,672.160461,289.724579,136.796616,206.915863,45.2677,353.983917,265.181671,62.333515,85.847481,703.133728,95.672699,418.562897
3,2010-01-07,142.236725,337.819672,481.012695,16.054960,53.326965,425.232696,473.382355,409.079987,1770.86731,341.851837,182.552383,85.970223,672.160461,289.724579,133.706726,209.240005,45.2677,353.983917,265.935547,62.381512,84.668106,703.133728,95.137444,431.439545
4,2010-01-08,144.094727,338.787170,478.753693,16.347025,53.540455,417.824463,473.382355,409.079987,1770.86731,341.851837,182.552383,84.762535,672.160461,289.724579,133.238586,207.948059,45.2677,353.983917,265.181671,62.033478,84.792007,703.133728,94.853218,440.510895


In [126]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier

In [127]:
class MyStrat(bt.Strategy):

  params = (('sma', 200),  # Simple moving average period
        ('lookback', 500),  # Lookback period for calculating returns
        ('rebal_cnt',20),)  #How many days do we want to rebalance after


  def __init__(self):
    self.cnt = 0
    self.SMA = []
    self.weights = []

    for d in self.datas:
      self.SMA.append(bt.indicators.SimpleMovingAverage(d.close,period=self.params.sma))

    self.selected_stocks = []

  def closingfn(self):
    for d in self.datas:
      self.close(data=d)

  def filterstocks(self):
    for i,d in enumerate(self.datas):
      if d.close[0] > self.SMA[i][0]:
        self.selected_stocks.append(d._name)


  def rebalance(self):
    #get weights by MVO
    df1 = df[df['Date'] <= datetime.datetime(self.data.datetime.date().year,self.data.datetime.date().month,self.data.datetime.date().day)]
    df1.set_index('Date',inplace=True)
    mu = mean_historical_return(df1)
    S = CovarianceShrinkage(df1).ledoit_wolf()
    ef = EfficientFrontier(mu, S)
    weights = ef.max_sharpe()

    wt = 0
    self.weights = np.zeros(len(self.datas))
    temp = 0;
    for stock in weights:
      self.weights[temp] = weights[stock]
      temp+=1

    temp=0
    for stock in weights:
      if stock in self.selected_stocks:
        wt+=weights[stock]
      else:
        self.weights[temp]=0
      temp+=1


    if wt == 0:
      self.weights = []
      self.selected_stocks = []
      return

    for i in range(len(self.weights)):
      self.weights[i]=self.weights[i]/wt

    weights_buy = self.weights
    portfolio_value = self.broker.getvalue()
    for i, d in enumerate(self.datas):
        if  start_dates[d._name] < self.data.datetime.date(0) - datetime.timedelta(days=self.params.lookback):
            # Placing an order to adjust the position size to match the opening price of the next day
            no_of_stocks=portfolio_value*weights_buy[i]/d.open[1]

            # round down
            no_of_stocks = int(no_of_stocks)
            self.buy(data=d, size=no_of_stocks)

    self.weights = []
    self.selected_stocks = []



  def next(self):
    if self.cnt % self.params.rebal_cnt == 0:
      self.closingfn()
      self.filterstocks()
      self.rebalance()

    self.cnt+=1



In [128]:
tickers = df2['ticker'].unique()
start_date = df2['date'].min()
end_date = df2['date'].max()
dates = pd.date_range(start=start_date, end=end_date, freq='D')
index = pd.MultiIndex.from_product([tickers, dates], names=['ticker', 'date'])
dummy_df = pd.DataFrame(index=index).reset_index()

# Merging the dummy DataFrame with the original DataFrame on the 'ticker' and 'date' columns
merged_df = pd.merge(dummy_df, df2, on=['ticker', 'date'], how='left')

# Forward filling missing values in the 'close' column with the next available value within each group of 'ticker'
merged_df['close'] = merged_df.groupby('ticker')['close'].fillna(method='bfill')

# Backward filling remaining missing values in the 'close' column with the previous available value within each group of 'ticker'
merged_df['close'] = merged_df.groupby('ticker')['close'].fillna(method='ffill')

# do the same for open, high, low
merged_df['open'] = merged_df.groupby('ticker')['open'].fillna(method='bfill')
merged_df['open'] = merged_df.groupby('ticker')['open'].fillna(method='ffill')

merged_df['high'] = merged_df.groupby('ticker')['high'].fillna(method='bfill')
merged_df['high'] = merged_df.groupby('ticker')['high'].fillna(method='ffill')

merged_df['low'] = merged_df.groupby('ticker')['low'].fillna(method='bfill')
merged_df['low'] = merged_df.groupby('ticker')['low'].fillna(method='ffill')

merged_df['open']=merged_df['open']
merged_df['high']=merged_df['high']
merged_df['low']=merged_df['low']

# Updating the original DataFrame with the merged DataFrame
df2 = merged_df

# Converting the 'date' column to a datetime object with the specified format again (this line is redundant)
df2['date'] = pd.to_datetime(df2['date'], format='%Y-%m-%d')

# Setting the 'date' column as the index of the DataFrame
df2 = df2.set_index('date')


In [129]:
cerebro = bt.Cerebro()

cerebro.addstrategy(MyStrat)

cerebro.broker.setcash(1000000)

for ticker in df2['ticker'].unique():
    # Creating a new DataFrame with only rows where the 'ticker' column is equal to the current ticker
    df3 = df2[df2['ticker'] == ticker]

    # Creating a data feed from the new DataFrame using the 'bt.feeds.PandasData' class
    df_feed = bt.feeds.PandasData(dataname=df3,
                                  open='open',
                                  close='close',
                                  high='high',
                                  low='low',
                                  volume=None,
                                  openinterest=None,
                                  datetime=None,
                                  fromdate=datetime.datetime(2010, 1, 1),
                                  todate=datetime.datetime(2020, 12, 31))

    # Adding the data feed to the Cerebro instance with a name equal to the current ticker
    cerebro.adddata(df_feed, name=ticker)

# add analyzers
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, riskfreerate=0.0)
cerebro.addanalyzer(bt.analyzers.Returns)
cerebro.addanalyzer(bt.analyzers.DrawDown)





In [ ]:
results = cerebro.run()

In [142]:
final_portfolio_value = cerebro.broker.getvalue()
maxdraw = results[0].analyzers.drawdown.get_analysis()['max']['drawdown']
cagr = results[0].analyzers.returns.get_analysis()['rnorm100']
sharpe = results[0].analyzers.sharperatio.get_analysis()['sharperatio']
cumret = ((final_portfolio_value/1000000) - 1)*100.0
print(f'Cumulative Returns: {cumret:.2f}')
print(f'Normalised Return (not CAGR) : {cagr:.2f} ')
print(f'Sharpe : {sharpe:.2f}')
print(f'Drawdown : {maxdraw:.2f}')

Cumulative Returns: 448.59
Normalised Return (not CAGR) : 11.90 
Sharpe : 0.65
Drawdown : 37.68


In [ ]:
#For detailed results
strat = results[-1]
portfolio_stats = strat.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
returns.index = returns.index.tz_convert(None)
qs.reports.html(returns, output='stats.html', title='Strategy Analysis')

In [ ]:
%matplotlib inline
# Setting the default figure size for matplotlib
plt.rcParams['figure.figsize'] = [15, 100]

# Updating the default font size for matplotlib
plt.rcParams.update({'font.size': 12})

# Plotting the results of the backtesting using the 'plot' method of the Cerebro instance
cerebro.plot()